<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install knmy
!pip install knmy

     |████████████████████████████████| 6.4MB 2.7MB/s 
  Created wheel for knmy: filename=knmy-1.0.0-py2.py3-none-any.whl size=5501 sha256=8b43392625d35d5915fe09450db2550940d1095efeb7c35c0561bd48f4c3ec64
  Stored in directory: /root/.cache/pip/wheels/a1/15/f8/75d11ed17dbbb170e1a83741e2d651f6655d2be905d3ed5616
Successfully built knmy


In [2]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [7]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
from knmy import knmy
import pandas as pd
import numpy as np

def knmi_get(start, end, stations=[240, 260]):
    
    # knmy.get_hourly_data returns a tuple with 4 items. Immediately index to [3] to get the df with weather variables. 
    knmi_data = knmy.get_hourly_data(stations=[240], start=start, end=end,
                            inseason=False, variables=['ALL'], parse=True)[3]
    
    # Rename columns
    cols = ['station', 'winddirection', 'windspeed_avg', \
            'windspeed_10m','windspeed_max', 'temperature', 'temperature_min', \
            'temperature_dewpoint', 'sunduration', 'sunradiation', \
            'precipitationduration', 'precipitation', 'airpressure', \
            'horizontalview', 'cloudcover', 'relativehumidity', 'weathercode', \
            'weathercodeindicator', 'mist', 'rain', 'snow', 'storm', 'ice']
    knmi_data.columns = cols
    
    # Drop useless columns
    knmi_data.drop(['winddirection', 'windspeed_10m', 'temperature_dewpoint', \
                    'horizontalview', 'cloudcover', 'weathercode', \
                    'weathercodeindicator'], axis = 1, inplace=True)
    # Drop first row since it's actually a header, then reset index
    knmi_data.drop([0], inplace=True)
    knmi_data.reset_index(drop=True, inplace=True)
    # Subtract one hour to make data in line with NDW
    knmi_data.hour = knmi_data.hour.astype(int) - 1
    # Remove columns with NA vals
    knmi_data.dropna(axis=1, inplace=True)
    # Make dataframe numeric
    knmi_data = knmi_data.apply(pd.to_numeric)
    # Remove negative precipitation values (-1 means 0.05mm or less)
    knmi_data['precipitation'] = \
        np.where(knmi_data['precipitation'] < 0, 0, knmi_data['precipitation'])
    # Make date col a datetime object
    knmi_data['date'] = pd.to_datetime(knmi_data['date'], yearfirst = True, 
                                                  format = '%Y%m%d').dt.date
    return knmi_data

In [0]:
import pandas as pd
traffic = pd.read_csv("data/df_imputed_week_shift_added_holiday_weekends_speed_limit_plus_10.csv", index_col = 0, parse_dates = True)

# New Section

In [5]:
*_, data = knmy.get_hourly_data(stations = [260], start = 2011010101, end = 2019123124, inseason = True, variables = ['WIND', 'TEMP'], parse = True)
# Drop first row (it's a duplicate header)
data.drop([0], inplace = True)
# Switch from end of hour (KNMI convention) to start of hour (NDW convention)
data.HH = (data.HH.astype(int) - 1).astype(str)
# Create datetime index
data.index = pd.to_datetime(data.YYYYMMDD.astype(str) + data.HH.str.zfill(2), format = "%Y%m%d%H", yearfirst = True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [12]:
join_traffic_weather(traffic_df, weather_df):
    freq = pd.infer_freq(traffic_df.index)
    weather = weather_df.copy().resample(freq).pad()
    


'T'

In [0]:
import datetime
begin = (traffic.index[0] + datetime.timedelta(hours = 1)).strftime("%Y%m%d%H")
end = (traffic.index[-1] + datetime.timedelta(hours = 1)).strftime("%Y%m%d%H")

In [25]:
end

'2019123000'